In [238]:
%%capture

# update or install the necessary libraries

%pip install langgraph
%pip install --upgrade \
    langchain==0.1.14 \
    langchain-core==0.1.31 \
    langchain-openai==0.1.3
%pip install --upgrade python-dotenv

#### 기본 환경 설정

In [239]:
from dotenv import load_dotenv

load_dotenv()

True

##### LLM 정의

In [240]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

##### 상태정의 

In [241]:
from typing import Annotated, List, TypedDict
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages  
from typing_extensions import Annotated, TypedDict, List

# class QuestionState(TypedDict):
#     histories: List[str]
#     messages: str


class QuestionState(TypedDict):
    # LangGraph 가 자동으로 append 해 주는 대화 메시지
    messages: Annotated[List[BaseMessage], add_messages]
    # 사용자가 따로 관리하고 싶은 이력
    histories: List[str]

##### Agent / Tool Prompt 정의 

In [242]:
from langchain_core.tools import StructuredTool
from pydantic import BaseModel, Field

question_agent_prompt = """
your are a answer agent only can do an operation if you need other then call other agent. Use Korean Language.
"""

mesage_context_prompt = """
This tool is a wrapper for answering messages from your question with one parameter

# VERY IMPORTANT
Your input to this tool MUST messages, state, histories and str, QuestionState, List type. 
"""

class MessageContext(BaseModel):
    messages: str = Field(..., description="message")

##### Agent 정의 

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_tool_calling_agent, AgentExecutor

def answer_question_agent_dict(state, agent, name):
    """ 
    Use conditional_edge to route with messages 
    """
    state["messages"] = [
        HumanMessage(content=f"""
당신은 전문적인 AI 어시스턴트입니다.  
- 아래 **대화 이력**에는 사용자가 과거에 보낸 모든 메시지와 당신의 답변이 시간순(오래된 → 최신)으로 들어 있습니다.  
- 최신 맥락을 파악해 가장 최근 질문에 답하세요.  
- 동일한 질문・요청이 반복되면, 과거 답변을 간략히 요약해 재사용하십시오.  
- 답변이 모호해질 때는 추가 정보를 정중히 요청하세요.  
- 코드·명령어·경로 등은 반드시 정확히 기재하고, 가능하면 공식 문서 출처를 덧붙이세요.  
- 한국어를 기본으로 사용하되, 사용자가 다른 언어로 질문하면 해당 언어로 답변하세요.  
- 200 단어(또는 1 000자) 이내로 간결하게 답하되, 단계별 설명·예제 코드·표를 적절히 활용하십시오.  
- 개인정보(실명·연락처 등)를 노출하지 말고, 보안 모범 사례를 준수하세요.
                     
**사용자 질문**을 대해서 답해주세요.                   

# 대화 이력:
{[humanMessage.content for humanMessage in state["histories"]]}

# 사용자 질문:
{[humanMessage.content for humanMessage in state["messages"]]}
        """)                
    ]
    print(f"{name} - agent call - before: {state}")

    result = agent.invoke(state)
    print(f"{name} - agent call - after: {state}")

    # 기본적으로 출력되는 메시지
    output_state = {
        "messages": [HumanMessage(content=result["output"], name=name)]
    }
    return output_state


def answer_question_agent(llm, tools, system_message: str):
    """Create an app."""
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a helpful AI assistant, collaborating with other assistants."
                " Use the provided tools to progress towards answering the question."
                " If you are unable to fully answer, that's OK, another assistant with different tools "
                " will help where you left off. Execute what you can to make progress."
                " If you or any of the other assistants have the final answer or deliverable,"
                " You have access to the following tools: {tool_names}.\n{system_message}",
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    prompt = prompt.partial(system_message=system_message)
    prompt = prompt.partial(tool_names=", ".join([tool.name for tool in tools]))
    agent = create_tool_calling_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

##### Graph Debug

In [244]:
# 디버깅 예시
def debug_state(state):
    print(f"Debuging 결과(State): {state}")
    # print(f"변수 값들: {state.values}")
    return state

##### Graph Build 

In [248]:
from langchain_core.messages import HumanMessage
from langgraph.constants import START, END
from langgraph.prebuilt import InjectedState
from langgraph.graph import StateGraph
from typing import Literal
import functools


def anwser_tool( 
        messages: str
    ) -> dict:
    
    print(f"anwser_tool - messages: {messages}")

    response = llm.invoke(
        [
            HumanMessage(
                content=messages
            )
        ]
    )
    print(f"anwser_tool - messages: {response}")
    
    # result = llm.invoke(messages)
    return response


agent_tools = [
                StructuredTool.from_function(
                    anwser_tool, 
                    name="anwser_tool", 
                    description=mesage_context_prompt, 
                    args_schema=MessageContext
                )
            ]

workflow_graph_builder = StateGraph(QuestionState)
    
workflow_graph_builder.add_node(
    "answer_question", 
    functools.partial(
        answer_question_agent_dict, 
        agent=answer_question_agent(
            llm=llm,
            tools=agent_tools, 
            system_message=question_agent_prompt
        ), 
        name="answer_question"
    )
)
workflow_graph_builder.add_node("debug", debug_state)

workflow_graph_builder.add_edge(START, "answer_question")
workflow_graph_builder.add_edge("answer_question", "debug")
workflow_graph_builder.add_edge("debug", END)

workflow_app = workflow_graph_builder.compile()

result = workflow_app.invoke({
    "histories" : [HumanMessage(content="""  
                    아래는 시대‧분야를 두루 아우르는 한국의 대표적 위인 100인을 간단히 나열한 목록입니다 (번호는 순위가 아니라 열거 순서입니다).
                   """),
                   HumanMessage(content="""
                   단군왕검, 고주몽, 박혁거세, 김수로왕, 근초고왕, 을지문덕, 연개소문, 계백, 김유신, 문무왕, 원효, 의상, 장보고, 설총, 강감찬, 서희, 최영, 최무선, 지눌, 이규보, 공민왕, 이성계, 정도전, 정몽주, 황희, 세종대왕, 장영실, 김종서, 한석봉, 이황(퇴계), 이이(율곡), 허준, 신사임당, 황진이, 김만덕, 유성룡, 권율, 이순신, 김시민, 곽재우, 김정호, 홍대용, 박지원(연암), 박제가, 정약용(다산), 김정희(추사), 김홍도(단원), 신윤복(혜원), 최치원, 허균, 김옥균, 박영효, 서재필(필립 제이슨), 이상설, 이회영, 안중근, 김구, 도산 안창호, 윤봉길, 이봉창, 유관순, 김좌진, 홍범도, 최재형, 신채호, 박은식, 한용운(만해), 윤동주, 이상화, 여운형, 조만식, 김규식, 최현배, 이승만, 김성수, 장면, 박정희, 김대중, 김영삼, 노무현, 문재인, 이병철, 구인회, 정주영, 이건희, 박태준, 유일한, 백남준, 김환기, 이중섭, 윤이상, 정명훈, 손기정, 차범근, 박지성, 손흥민, 김연아, 우장춘, 장지연, 안익태""")],
    "messages": [HumanMessage(content="위의 사람중에서 무작위로 3명만 다시 정리해줘")]
})

print(f"""workflow_app.invoke 결과 : {result["messages"][1]}""")

answer_question - agent call - before: {'messages': [HumanMessage(content="\n당신은 전문적인 AI 어시스턴트입니다.  \n- 아래 **대화 이력**에는 사용자가 과거에 보낸 모든 메시지와 당신의 답변이 시간순(오래된 → 최신)으로 들어 있습니다.  \n- 최신 맥락을 파악해 가장 최근 질문에 답하세요.  \n- 동일한 질문・요청이 반복되면, 과거 답변을 간략히 요약해 재사용하십시오.  \n- 답변이 모호해질 때는 추가 정보를 정중히 요청하세요.  \n- 코드·명령어·경로 등은 반드시 정확히 기재하고, 가능하면 공식 문서 출처를 덧붙이세요.  \n- 한국어를 기본으로 사용하되, 사용자가 다른 언어로 질문하면 해당 언어로 답변하세요.  \n- 200 단어(또는 1 000자) 이내로 간결하게 답하되, 단계별 설명·예제 코드·표를 적절히 활용하십시오.  \n- 개인정보(실명·연락처 등)를 노출하지 말고, 보안 모범 사례를 준수하세요.\n                     \n**사용자 질문**을 대해서 답해주세요.                   \n\n# 대화 이력:\n['  \\n                    아래는 시대‧분야를 두루 아우르는 한국의 대표적 위인 100인을 간단히 나열한 목록입니다 (번호는 순위가 아니라 열거 순서입니다).\\n                   ', '\\n                   단군왕검, 고주몽, 박혁거세, 김수로왕, 근초고왕, 을지문덕, 연개소문, 계백, 김유신, 문무왕, 원효, 의상, 장보고, 설총, 강감찬, 서희, 최영, 최무선, 지눌, 이규보, 공민왕, 이성계, 정도전, 정몽주, 황희, 세종대왕, 장영실, 김종서, 한석봉, 이황(퇴계), 이이(율곡), 허준, 신사임당, 황진이, 김만덕, 유성룡, 권율, 이순신, 김시민, 곽재우, 김정호, 홍대용, 박지원(연암), 박제가, 정약용(다산), 김정희(추사), 김홍도(단원), 신윤복(혜원), 최치원, 허